In [ ]:
# Defining the file name from github
filename = '../Dataset/ad.data'

In [ ]:
import pandas as pd
# Loading the data using pandas

adData = pd.read_csv(filename,sep=",",header = None,error_bad_lines=False)
adData.head()

In [ ]:
# Seperating the dependent and independent variables
# Preparing the X variables
X = adData.loc[:,0:1557]
print(X.shape)
# Preparing the Y variable
Y = adData[1558]
print(Y.shape)


In [ ]:
import numpy as np
# Replacing special characters in first 3 columns which are of type object
for i in range(0,3):
  X[i] = X[i].str.replace("?", 'NaN').values.astype(float)
# Replacing special characters in the remaining columns which are of type integer
for i in range(3,1557):
  X[i] = X[i].replace("?", 'NaN').values.astype(float)  
# Imputing the 'nan'  with mean of the values
for i in range(0,1557):
  X[i] = X[i].fillna(X[i].mean())

In [ ]:
# Normalising the data sets
# Normalising data
from sklearn import preprocessing
# Creating the scaling function
minmaxScaler = preprocessing.MinMaxScaler()
X_tran = pd.DataFrame(minmaxScaler.fit_transform(X))
X_tran.head()

In [ ]:
# Creating a high dimension data set
X_hd = pd.DataFrame(pd.np.tile(X_tran, (1, 2)))

print(X_hd.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# Defining the Classification function
backModel = LogisticRegression()
# Reducing dimensionality to 250 features for backward elimination model
rfe = RFE(backModel, 250)


In [ ]:
# Fitting the rfe for selecting the top 250 features
import time
t0 = time.time()
rfe = rfe.fit(X_hd, Y)
t1 = time.time()
print("Backward Elimination time:", round(t1-t0, 3), "s")

In [ ]:
# Getting the indexes of the features used
rfe.get_support(indices = True)

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_hd, Y, test_size=0.3, random_state=123)

print('Training set shape',X_train.shape)

print('Test set shape',X_test.shape)

In [ ]:
# Transforming both train and test sets

X_train_tran = rfe.transform(X_train)

X_test_tran = rfe.transform(X_test)

print("Training set shape",X_train_tran.shape)

print("Test set shape",X_test_tran.shape)

In [ ]:
# Fitting the logistic regression model 
import time
# Defining the LogisticRegression function
RfeModel = LogisticRegression()
# Starting a timing function
t0=time.time()
# Fitting the model
RfeModel.fit(X_train_tran, y_train)
# Finding the end time 

print("Total training time:", round(time.time()-t0, 3), "s")





In [ ]:
# Predicting on the test set and getting the accuracy
pred = RfeModel.predict(X_test_tran)

print('Accuracy of Logistic regression model after backward elimination: {:.2f}'.format(RfeModel.score(X_test_tran, y_test)))



In [ ]:
from sklearn.metrics import confusion_matrix
confusionMatrix = confusion_matrix(y_test, pred)
print(confusionMatrix)

In [ ]:
from sklearn.metrics import classification_report
# Getting the Classification_report
print(classification_report(y_test, pred))